# Desafío Data Science Latam

## Importa librerías

In [1]:
import pandas as pd

## Carga archivo de datos

In [3]:
latamDF = pd.read_csv('dataset_SCL.csv', parse_dates=['Fecha-I','Fecha-O'])
latamDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Fecha-I    68206 non-null  datetime64[ns]
 1   Vlo-I      68206 non-null  object        
 2   Ori-I      68206 non-null  object        
 3   Des-I      68206 non-null  object        
 4   Emp-I      68206 non-null  object        
 5   Fecha-O    68206 non-null  datetime64[ns]
 6   Vlo-O      68205 non-null  object        
 7   Ori-O      68206 non-null  object        
 8   Des-O      68206 non-null  object        
 9   Emp-O      68206 non-null  object        
 10  DIA        68206 non-null  int64         
 11  MES        68206 non-null  int64         
 12  AÑO        68206 non-null  int64         
 13  DIANOM     68206 non-null  object        
 14  TIPOVUELO  68206 non-null  object        
 15  OPERA      68206 non-null  object        
 16  SIGLAORI   68206 non-null  object       

C:\Users\an2fe\anaconda3v2\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Pregunta 1
### ¿Cómo se distribuyen los datos? ¿Qué te llama la atención o cuál es tu conclusión sobre esto?

Los datos estan claramente diferenciados entre programados y operado, lo cual facilita determinar cuáles vuelos se retrasaron. En ambos casos (programado y operado), se incluye información en la fecha y hora, número de vuelo, ciudades de origen y destino, y código de la aerolinea.

Adicionalmente se incluye columnas individuales para el dia, mes, y año en que los vuelos fueron operados. Inicialmente esta información parece redundante con la columna "Fecha-O". Adicionalmente se incluye el día de la semana, el cuál puede ser relevante para predecir el retraso de un vuelo.

Una columna con el tipo de vuelo (Nacional o Internacional), la cuál también puede ser de importancia para la predicción.

Finalmente, se tienen tres columnas con los nombres completos de la aerolinea operadora, ciudad de origen y ciudad de destino. Esta información es redundante con las columnas "Ori-O", "Des-O", "Emp-O".  

A continuación se realizará una exploración de los datos para conocer más en detalle la distribución de los datos en el tiempo.